## DSI-06 Homework 6: ANSWERS
From Chapter 7, found on page 327 of ISLP

*This question uses the variables `dis` (the weighted mean of distances to five Boston employment centers) and `nox` (nitrogen oxides concentration in parts per 10 million) from the `Boston` data. We will treat `dis` as the predictor and `nox` as the response.*

In [ ]:
# Import libraries and specific objects
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
import statsmodels.api as sm
import random
import patsy 
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize)
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from patsy import dmatrix
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from statsmodels.stats.anova import anova_lm

# Load Default data
Boston = load_data('Boston')
Boston

(a)  Use the `poly()` function to fit a cubic polynomial regression to predict nox using dis. Report the regression output, and plot the resulting data and polynomial fits.

In [ ]:
# Create polynomial features using numpy
Boston['dis_poly'] = np.power(Boston['dis'], 3)

# Fit a cubic polynomial regression
poly_fit = sm.OLS.from_formula('nox ~ dis_poly', data=Boston).fit()

# Print regression output
print(poly_fit.summary())

In [ ]:
# Generate a range of 'dis' values for predictions
dis_range = np.linspace(min(Boston['dis']), max(Boston['dis']), 100)
dis_poly_range = np.power(dis_range, 3)

# Predict response and standard errors
pred = poly_fit.get_prediction({'dis_poly': dis_poly_range})
conf_int = pred.conf_int()

# Create a dataframe with predictions and confidence intervals
predictions = pd.DataFrame({
    'DIS': dis_range,
    'NOX': pred.predicted_mean,
    'upper': conf_int[:, 1],
    'lower': conf_int[:, 0]
})

# Plot the observed values, predicted values, and confidence intervals
plt.scatter(Boston['dis'], Boston['nox'], color='black', alpha=0.2, label='Observed Values')
plt.plot(predictions['DIS'], predictions['NOX'], color='blue', linewidth=1.5, label='Predicted Values')
plt.fill_between(predictions['DIS'], predictions['lower'], predictions['upper'], alpha=0.2, label='95% Confidence Interval')
plt.xlabel('dis')
plt.ylabel('nox')
plt.legend()
plt.show()

(b) Plot the polynomial fits for a range of different polynomial degrees (say, from 1 to 10), and report the associated residual sum of squares.

In [ ]:
# Set the range of polynomial degrees
range_poly = range(1, 11)

# Initialize a figure for plotting
plt.figure(figsize=(12, 8))

# Iterate over polynomial degrees
for degrees in range_poly:
    # Create polynomial features using patsy
    formula = f'nox ~ bs(dis, degree={degrees}, include_intercept=False, df=11)'
    y, X = patsy.dmatrices(formula, Boston, return_type='dataframe')

    # Fit a polynomial regression
    poly_fit = sm.OLS(y, X).fit()

    # Generate a range of 'dis' values for predictions
    dis_range = np.linspace(min(Boston['dis']), max(Boston['dis']), 100)
    X_range = patsy.build_design_matrices([X.design_info], {'dis': dis_range})[0]

    # Predict response
    nox_pred = poly_fit.predict(X_range)

    # Plot the data and polynomial fit
    plt.scatter(Boston['dis'], Boston['nox'], alpha=0.2, label='Observed Data')
    plt.plot(dis_range, nox_pred, label=f'{degrees} degrees Polynomial Fit')

    # Calculate and print residual sum of squares
    errors = np.sum(np.square(poly_fit.resid)) / (len(Boston) - len(poly_fit.params))
    print(f"Residual sum of squares for {degrees} degrees: {errors}")

# Add labels and legend to the plot
plt.xlabel('dis')
plt.ylabel('nox')
plt.legend()
plt.title('Polynomial Fits for Different Degrees')
plt.show()

(c)  Perform cross-validation or another approach (hint: *anova*) to select the optimal degree for the polynomial, and explain your results.

In [ ]:
from sklearn.linear_model import LinearRegression

np.random.seed(16)
range_poly = np.arange(1, 11)
cv_error = np.empty(10)

for degrees in range_poly:
    poly_model = make_pipeline(PolynomialFeatures(degrees), LinearRegression())
    cv_error[degrees - 1] = -np.mean(cross_val_score(poly_model, Boston[['dis']], Boston['nox'], scoring='neg_mean_squared_error', cv=10))

best_degree = np.argmin(cv_error) + 1
print(f"The best polynomial degree is: {best_degree}")

Alternatively, we can use an Anova to compare the 10 degrees

In [ ]:
degrees = np.arange(1, 11)

# Fit polynomial models and store them in a list
poly_fits = [sm.OLS(Boston['nox'], sm.add_constant(PolynomialFeatures(degree).fit_transform(Boston[['dis']])))
             .fit() for degree in degrees]

# Perform ANOVA
anova_table = sm.stats.anova_lm(*poly_fits)
print(anova_table)

The p-values from the ANOVA indicate that we can reject the null hypothesis when compare model 1 to
model 2 and model 2 to model 3. This means that both the linear and quadratic models are not sufficient to
explain the data. The p-value for the comparison of model 3 and 4 is not significant so we could say that
a 3rd degree polynomial is sufficient to explain the data over the 4th. In this case, 3rd degree polynomial
would be sufficient to explain the data, similar to what was found using CV.

(d) Use the `bs()` function to fit a regression spline to predict nox using dis. Report the output for the fit using four degrees of freedom. How did you choose the knots? Plot the resulting fit.

In [ ]:
# Fit a cubic B-spline with 4 degrees of freedom
spline_formula = 'nox ~ bs(dis, df=4, degree=3, include_intercept=True) - 1'
y, X = patsy.dmatrices(spline_formula, Boston, return_type='dataframe')

# Fit the spline model
spline_fit = sm.OLS(y, X).fit()

# Print summary
print(spline_fit.summary())

(e)  Now fit a regression spline for a range of degrees of freedom, and plot the resulting fits and report the resulting RSS. Describe the results obtained.

In [ ]:
# Set the range of spline degrees
range_splines = range(3, 11)

for spline_degree in range_splines:
    # Fit a spline model
    formula = f'nox ~ bs(dis, df={spline_degree}) - 1'
    spline_fit = sm.OLS.from_formula(formula, data=Boston).fit()

    # Generate a range of 'dis' values for predictions
    dis_range = np.linspace(min(Boston['dis']), max(Boston['dis']), 100)
    X_range = pd.DataFrame({'dis': dis_range})
    
    # Predict response
    pred_results = spline_fit.get_prediction(X_range)
    pred = pred_results.predicted_mean
    conf_int = pred_results.conf_int()

    # Calculate and print residual sum of squares
    errors = np.sum(np.square(spline_fit.resid)) / (len(Boston) - len(spline_fit.params))
    print(f"Summary of Model fit for {spline_degree} degrees of freedom spline regression")
    print(spline_fit.summary())
    print(f"Residual sum of squares: {errors}")
    print("--------------------------------------------------------------")

    # Plot the data and spline fit with confidence interval
    plt.scatter(Boston['dis'], Boston['nox'], alpha=0.2, label='Observed Data')
    plt.plot(dis_range, pred, label=f'{spline_degree} degrees of freedom Spline Fit', color='blue')
    plt.fill_between(dis_range, conf_int[:, 0], conf_int[:, 1], color='blue', alpha=0.2, label='95% Confidence Interval')

    # Add labels and legend to the plot
    plt.xlabel('dis')
    plt.ylabel('nox')
    plt.legend()
    plt.title(f'Spline Fit with {spline_degree} Degrees of Freedom and Confidence Interval')
    plt.show()

(f)   Perform cross-validation or another approach in order to select the best degrees of freedom for a regression spline on this data. Describe your results.

*Use Anova to compare models*

In [ ]:
# Fit spline models
spline_fit3 = sm.OLS.from_formula('nox ~ bs(dis, df=3) - 1', data=Boston).fit()
spline_fit4 = sm.OLS.from_formula('nox ~ bs(dis, df=4) - 1', data=Boston).fit()
spline_fit5 = sm.OLS.from_formula('nox ~ bs(dis, df=5) - 1', data=Boston).fit()
spline_fit6 = sm.OLS.from_formula('nox ~ bs(dis, df=6) - 1', data=Boston).fit()
spline_fit7 = sm.OLS.from_formula('nox ~ bs(dis, df=7) - 1', data=Boston).fit()
spline_fit8 = sm.OLS.from_formula('nox ~ bs(dis, df=8) - 1', data=Boston).fit()
spline_fit9 = sm.OLS.from_formula('nox ~ bs(dis, df=9) - 1', data=Boston).fit()
spline_fit10 = sm.OLS.from_formula('nox ~ bs(dis, df=10) - 1', data=Boston).fit()

# Perform ANOVA
anova_results = anova_lm(spline_fit3, spline_fit4, spline_fit5, spline_fit6, spline_fit7, spline_fit8, spline_fit9, spline_fit10)
print(anova_results)